In [137]:
import pandas as pd
import sys

In [138]:
tbl = pd.read_csv("test.smbtbl", delimiter='|')
tbl.set_index("MLo", inplace=True)
print(tbl.columns)
tbl

Index(['Type', 'Lenght', 'Name', 'Scope', 'LineNumbers'], dtype='object')


,Type,Lenght,Name,Scope,LineNumbers
MLo,,,,,
2,int,0,y,main,8 9 10
1,int,0,x,main,8 9 10
4,void,0,output,global,10
3,int,0,input,global,9 9
0,void,0,main,global,7
0,int,0,gcd,global,1 4 10
3,int,0,v,gcd,1 3 4 4 4
2,int,0,u,gcd,1 3 4 4


In [139]:
file_path = 'test.quad'
commands = []
with open(file_path, 'r') as file:
    for line in file:
        quad = line.strip('(')
        quads = quad[:-2].split(', ')
        commands.append(quads)
print(commands[0][3])

-


In [140]:

usedIndexes = []
def fixIndex(line):
    global usedIndexes
    tobeUsed = []
    for command in commands[line + 1:]:
        # index = command[1].strip("$t")
        for i in range(1,len(command)):
            if "$t" in command[i]:
                #print(command[i])
                if command[i][2:] in usedIndexes and command[i][2:] not in tobeUsed:
                    tobeUsed.append(command[i][2:])
    print(usedIndexes)
    print("\n")
    usedIndexes = list(set(usedIndexes) - set(set(usedIndexes) - set(tobeUsed)))

# tobeUsed

In [141]:
tbl.index[tbl["Name"] == "main"][0]

0

In [142]:
def getIndex(reg, indice, indice2):
    if "$t" in commands[indice][indice2]:
        index = int(reg[2:])
        print("index = ",index)
        if index == 0:
            print("old = ",reg, "|returned = ",commands[indice][indice2])
            return commands[indice][indice2]
        if index < 29:
            print("old = ",reg, "|returned = ",commands[indice][indice2])
            return commands[indice][indice2]
        for num in range(1, 29):
            print(str(num),usedIndexes)
            if str(num) not in usedIndexes:
                #print(i, " Esta disponivel\n")
                for i in range(0,len(commands)):
                    for j in range(1,len(commands[i])):
                        if "$t" in commands[i][j]:
                            print("for ",commands[i][j], reg)
                            if commands[i][j] == reg:
                                commands[i][j] = "$t"+str(num)
                                print("Novo reg = ","$t"+str(num))
                                break
                print("old = ",reg, "|returned = ","$t"+str(num))
                print("REMOVED = ",index, "|appended = ","$t"+str(num))
                print(usedIndexes)
                usedIndexes.remove(str(index))
                usedIndexes.append(str(num))
                return "$t"+str(num)
        print("\n----------------------------\nFATAL ERROR: NOT ENOUGH REGISTERS!!!\n----------------------------\n")
    return "FATAL ERROR: NOT ENOUGH REGISTERS!!!"

In [143]:
tbl[(tbl['Name'] == 'v') & (tbl['Scope'] == 'gcd')].index[0]

3

In [144]:
#addi beq bne bgt blt lw sw j jal (jar) add sub div mult and or
assembly = open("test.asm", 'w')
scope = " "
assembly.write("j "+str(tbl.index[tbl["Name"] == "main"][0])+"\n")
for i, command in enumerate(commands):
    for reg in command[1:]:
        if "$t" in reg and reg[2:] not in usedIndexes:
            usedIndexes.append(reg[2:])
    print(command)
    match command[0]:
        case 'FUNC':
            scope = command[2]
            assembly.write("addi $t30 $t30 1\n")
            assembly.write("sw $t31 $t29 1"+"\n")

    #
        case 'ARG':
            assembly.write("addi $t30 $t30 1\n")
        case 'LOAD':
            #assembly.write(command[2] + " " + scope+"\n")
            assembly.write("lw "+getIndex(command[1],i,1)+" "+"$t29 "+str(tbl[(tbl['Name'] == command[2]) & (tbl['Scope'] == scope)].index[0])+"\n")
    #
    #     case 'GOTO':
    #         print("eh GOTO")
    #     case 'LAB':
    #         print("eh LAB")
    #     case 'WHILE':
    #         print("eh WHILE")
    #     case 'RET':
    #         print("eh RET")
    #     case 'END':
    #         print("eh END")
        case 'CALL':
            assembly.write("addi "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+command[3]+"\n")
        case 'ADDI':
            #command[2] is always $t0
            assembly.write("addi "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+command[3]+"\n")
        case 'ALLOC':
            assembly.write("addi $t30 $t30 1\n")
    #     case 'STORE':
    #         print("eh STORE")
    #     case 'PARAM':
    #         print("eh PARAM")
        case 'SOM':
            assembly.write("add "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'SUB':
            assembly.write("sub "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'MULT':
            assembly.write("mult "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
        case 'DIV':
            assembly.write("div "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
    #     case 'LT':
    #         print("eh LT")
    #     case 'LET':
    #         print("eh LET")
    #     case 'GT':
    #         print("eh GT")
    #     case 'GET':
    #         print("eh GET")
        case 'COMP':
            assembly.write("comp "+getIndex(command[1],i,1)+" "+getIndex(command[2],i,2)+" "+getIndex(command[3],i,3)+"\n")
    #     case 'NEQ':
    #         print("eh NEQ")
    #     case 'EQ':
    #         print("eh EQ")
    fixIndex(i)
#print(usedIndexes)
assembly.close()

['FUNC', 'inteiro', 'gcd', '-']
[]


['ARG', 'inteiro', 'u', 'global']
[]


['ARG', 'inteiro', 'v', 'global']
[]


['LOAD', '$t1', 'v', '-']
index =  1
old =  $t1 |returned =  $t1
['1']


['ADDI', '$t2', '$t0', '0']
index =  2
old =  $t2 |returned =  $t2
index =  0
old =  $t0 |returned =  $t0
['1', '2', '0']


['COMP', '$t3', '$t1', '$t2']
index =  3
old =  $t3 |returned =  $t3
index =  1
old =  $t1 |returned =  $t1
index =  2
old =  $t2 |returned =  $t2
['1', '2', '3']


['IFF', '$t3', 'L0 -']
['3']


['LOAD', '$t4', 'u', '-']
index =  4
old =  $t4 |returned =  $t4
['4']


['RET', '$t4', '-', '-']
['4']


['GOTO', 'L1', '-', '-']
[]


['LAB', 'L0', '-', '-']
[]


['LOAD', '$t5', 'v', '-']
index =  5
old =  $t5 |returned =  $t5
['5']


['PARAM', '$t5', '-', '-']
['5']


['LOAD', '$t6', 'u', '-']
index =  6
old =  $t6 |returned =  $t6
['6']


['LOAD', '$t7', 'u', '-']
index =  7
old =  $t7 |returned =  $t7
['6', '7']


['LOAD', '$t8', 'v', '-']
index =  8
old =  $t8 |returned =  $t8
['6

In [145]:
for command in commands:
    print(command)

['FUNC', 'inteiro', 'gcd', '-']
['ARG', 'inteiro', 'u', 'global']
['ARG', 'inteiro', 'v', 'global']
['LOAD', '$t1', 'v', '-']
['ADDI', '$t2', '$t0', '0']
['COMP', '$t3', '$t1', '$t2']
['IFF', '$t3', 'L0 -']
['LOAD', '$t4', 'u', '-']
['RET', '$t4', '-', '-']
['GOTO', 'L1', '-', '-']
['LAB', 'L0', '-', '-']
['LOAD', '$t5', 'v', '-']
['PARAM', '$t5', '-', '-']
['LOAD', '$t6', 'u', '-']
['LOAD', '$t7', 'u', '-']
['LOAD', '$t8', 'v', '-']
['DIV', '$t9', '$t7', '$t8']
['LOAD', '$t10', 'v', '-']
['MULT', '$t11', '$t9', '$t10']
['SUB', '$t12', '$t6', '$t11']
['PARAM', '$t12', '-', '-']
['CALL', '$t13', 'gcd', '2']
['RET', '$t13', '-', '-']
['GOTO', 'L1', '-', '-']
['LAB', 'L1', '-', '-']
['END', 'gcd', '-', '-']
['FUNC', 'void', 'main', '-']
['ALLOC', 'x', 'main', '-']
['ALLOC', 'y', 'main', '-']
['CALL', '$t14', 'input', '0']
['STORE', 'x', '$t14', '-']
['CALL', '$t15', 'input', '0']
['STORE', 'y', '$t15', '-']
['LOAD', '$t16', 'x', '-']
['PARAM', '$t16', '-', '-']
['LOAD', '$t17', 'y', '-']
